<h1>IS4242 Group Project</h1>

<b>Import necessary libraries</b>

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor


Ensure that you are in the root folder of all the fold folders and target files
read_text(fold_name):
    fold_name: this is the name of the fold you want to read ALL patient files of. It will be read into a 2 dimensional
    list. If you would like to retrieve just the first patient instead, you will need to change the line 
    "txt_all.extend(txt[1:])" to "txt_all.append(txt[1:])" and you will be to use "read_text(fold1.txt)[0]" to retrieve
    the relevant patient's data
read_ans(file_name):
    file_name: this is the name of the file you want to read ALL targets of. It will be read into a 2 dimensional
    list. To retrieve the first patient's target: read_ans(ans.csv)[0]
put_single_into_dataframe(txt): This functions takes in 2 dimensional list ie the output of read_text(fold1.txt) 
put_multiple_into_dataframe(txt): Multiple is for using it with the output of read_text after you wanted to change it to append


In [2]:
def read_text(fold_name):
    txt_all = list()
    for f in os.listdir(fold_name): # for each file in the directory
        if f.endswith(".txt"):
            with open(os.path.join(fold_name, f), 'r') as fp: # open each file
                txt = fp.readlines() # read inside the file
                recordid = txt[1].rstrip('\n').split(',')[-1] # get recordid
                txt = [[int(recordid)] + t.rstrip('\n').split(',') for t in txt] # preface each row with the recordid as all patients are 1 file
                txt_all.extend(txt[1:]) # skip the parameter list
    return txt_all

def read_ans(file_name):
    txt_all = list()
    with open(file_name, 'r') as fp: # opens the csv file
        txt = fp.readlines() 
    for i in range(1, len(txt)): # similar to above read_text
        record_id, length_of_stay, hospital_death = txt[i].rstrip('\n').split(',')
        txt_all.append([record_id, length_of_stay, hospital_death])
    return txt_all

def put_multiple_into_dataframe(txt_all):
    df = pd.DataFrame()
    for i in txt_all:
        df2 = pd.DataFrame(i, columns=['recordid', 'time', 'parameter', 'value'])
        df = df.append(df2, ignore_index=True)
    return df

def put_single_into_dataframe(txt_all):
    df = pd.DataFrame(txt_all, columns=['recordid', 'time', 'parameter', 'value'])
    return df

def get_X_add_ready(X_add, stat):
    X_add = X_add.reset_index()
    X_add = X_add.pivot(index='recordid', columns='parameter', values='value')
    X_add = X_add.drop(stat_feat, axis = 1) 
#     X_add = X_add.drop(['RecordID'], axis = 1) 
    X_add.columns = [x+stat for x in X_add.columns]
    X_add = X_add.reset_index()
    return X_add

In [3]:
df_feat = pd.DataFrame()
numberOfFolds = 0
for i, name in enumerate(["Fold1"]): # what folds do you want to use?
    string = "../Project_Data/"
    string += name
    df_feat = df_feat.append(put_single_into_dataframe(read_text(string)))
    numberOfFolds = (i+1)
numberOfRows = numberOfFolds*1000
df_feat.head()

,recordid,time,parameter,value
0,132539,00:00,RecordID,132539
1,132539,00:00,Age,54
2,132539,00:00,Gender,0
3,132539,00:00,Height,-1
4,132539,00:00,ICUType,4


In [4]:
# Reading Target
df_target = pd.DataFrame(read_ans('../Project_Data/Fold1_Outcomes.csv'), columns=['recordid', 'days_in_hospital', 'mortality'])
df_target.head()

,recordid,days_in_hospital,mortality
0,132539,5,0
1,132540,8,0
2,132541,19,0
3,132543,9,0
4,132545,4,0


In [5]:

bin_feat = ['MechVent']
num_feat = ['Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol',
           'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT',
           'HR', 'K', 'Lactate', 'Mg', 'MAP', 'NA', 'NIDiasABP', 'NIMAP',
           'NISysABP', 'PaCO2', 'PaO2', 'pH', 'Platelets', 'RespRate', 'SaO2',
           'SysABP', 'Temp', 'Tropl', 'TropT', 'Urine', 'WBC', 'Weight']

print("Number of record ids:", len(df_feat['recordid'].unique()))
unique_count = df_feat['parameter'].value_counts()/numberOfRows
print(unique_count)

Number of record ids: 1000
HR             57.027
MAP            36.092
SysABP         35.979
DiasABP        35.955
Urine          34.208
Weight         33.679
NISysABP       24.457
NIDiasABP      24.424
NIMAP          24.088
Temp           21.204
GCS            15.214
RespRate       13.775
FiO2            7.815
MechVent        7.596
pH              5.770
PaO2            5.496
PaCO2           5.490
HCT             4.626
K               3.708
Creatinine      3.573
Platelets       3.566
BUN             3.547
HCO3            3.479
Mg              3.468
Na              3.462
Glucose         3.338
WBC             3.286
SaO2            1.985
Lactate         1.924
Height          1.000
Gender          1.000
Age             1.000
ICUType         1.000
RecordID        1.000
Bilirubin       0.858
ALT             0.857
AST             0.857
ALP             0.833
Albumin         0.617
TroponinT       0.566
TroponinI       0.130
Cholesterol     0.077
Name: parameter, dtype: float64


<h2>Analysis of Features</h2>
<p>The data above shows the average number of times a variable observed per patient. Based on the data above and the feature description we classify the features into these categories:
<ul>
    <li>General Descriptors (static data) that are collected when the patient is admitted to the ICU. Weight is not included as weight are measured multiple times as a time series data. Each of the descriptors will be included as a feature into the model.</li>
    <li>Rare features: measured on average less than one time per patient (less than 1.0). We use the <u>existence</u> of these measurements for each patient as a feature.</li>
    <li>Features that measured often or more that one time per patient (more than 1.0). Calculate the hourly average of each measurements and put them into 48 columns. <i>Example, average HR on the first hour to HR_1, average HR on the second hour to HR_2, and so on.</i></li>
</ul>
</p>


In [6]:
stat_feat = ['Age', 'Gender', 'Height', 'ICUType', 'RecordID'] #General Descriptors
rare_feat = []
nor_feat = []
for index, value in unique_count.items():
    if value < 1.0:
        rare_feat.append(index)
    elif index not in stat_feat:
        nor_feat.append(index)
rare_feat.append("MechVent")
print("Rare features", rare_feat)
print("Normal features", nor_feat)

Rare features ['Bilirubin', 'ALT', 'AST', 'ALP', 'Albumin', 'TroponinT', 'TroponinI', 'Cholesterol', 'MechVent']
Normal features ['HR', 'MAP', 'SysABP', 'DiasABP', 'Urine', 'Weight', 'NISysABP', 'NIDiasABP', 'NIMAP', 'Temp', 'GCS', 'RespRate', 'FiO2', 'MechVent', 'pH', 'PaO2', 'PaCO2', 'HCT', 'K', 'Creatinine', 'Platelets', 'BUN', 'HCO3', 'Mg', 'Na', 'Glucose', 'WBC', 'SaO2', 'Lactate']


In [7]:
# your code to produce test and train data
df = df_feat.copy()

<h2>Creation of Data Matrices</h2>
<p>We create 3 different matrices to convert temporal data into a matrix that is a single feature vector per patient </p>
<ul> 
    <li>First, in the cell below, we create a matrix that generalises a patient's attributes across the whole 48 hours, such as his max BUN measurement over the 48 hours. </li>
    <li>Second, Junhao what is your matrix?</li>
    <li></li>
</ul>

In [8]:
def preprocess_x_for_design_matrix_1(df_feat):
    non_bin_feat.clear()
    # your code to produce test and train data
    df = df_feat.copy()

    df['value'] = pd.to_numeric(df['value'])
    temp_df = df.loc[df['time'] == '00:00', :].copy() # get all the variables at time 0
    temp_df = temp_df.loc[temp_df['parameter'].isin(stat_feat)] # prune the dataframe to only those static variables
    temp_df = temp_df.pivot(index='recordid', columns='parameter', values='value') 
    temp_df = temp_df.reset_index()
    for i in temp_df: # for loop to change all the -1 values for static variables into np.nan
        idx = temp_df.index[temp_df[i] == -1].tolist()
        for j in idx:
            temp_df.loc[j, i] = np.nan
    final_df = temp_df.copy()

#     Dealing with rare_feat
    d = df_feat.groupby(['recordid', 'parameter'])[['value']].count()
    def specialFeature(special):
        id = []
        for index, row in d.iterrows():
            if index[1] == special:
                id.append(index[0])
        return id
    for x in rare_feat:
        id = specialFeature(x)
        final_df[x] = 0
        for i in id:
            for row in final_df.index:
                if row == i:
                    final_df.loc[row, x] = 1
    final_df = final_df.drop(["RecordID"],axis=1)

    # Getting the different attributes
    pd.set_option('display.max_columns', 500)
    temp_df = df.drop(df.index[df['parameter'].isin(rare_feat)].tolist())
    temp_df = temp_df.groupby(['recordid', 'parameter'])[['value']]
    for i in ['min', 'max', 'mean']: # the different parameters we will use
        if (i=='min'):
            X_add = temp_df.min() # get the min of each parameter
            X_add = get_X_add_ready(X_add, '_min')
            final_df = final_df.merge(X_add, left_on='recordid', right_on='recordid') # merge the min of the parameters to the final dataframe
        elif (i=='max'):
            X_add = temp_df.max() # get the max of each parameter
            X_add = get_X_add_ready(X_add, '_max')
            final_df = final_df.merge(X_add, left_on='recordid', right_on='recordid') # merge the min of the parameters to the final dataframe
        elif (i=='mean'):
            X_add = temp_df.mean() # get the mean of each parameter
            X_add = get_X_add_ready(X_add, '_mean')
            final_df = final_df.merge(X_add, left_on='recordid', right_on='recordid') # merge the min of the parameters to the final dataframe

    # dealing with ICUType categorical
    one_hot = pd.get_dummies(final_df['ICUType'])
    meaning_of_icu_types = {1:'Coronary Care Unit', 2: 'Cardiac Surgery Recovery Unit', 3: 'Medical ICU', 4: 'Surgical ICU'}
    one_hot.columns = [meaning_of_icu_types[x] for x in one_hot.columns]
    final_df = final_df.merge(one_hot, left_index=True, right_index=True)
    final_df = final_df.drop('ICUType', axis=1)
    
    # Extreme height values is set to np.nan
    for index, row in final_df.iterrows():
        if row["Height"] < 40 or row["Height"] > 210:
            row["Height"] = np.nan


    # Drop recordID column
    final_df = final_df.drop("recordid", axis=1)
            
    # Creating non binary column list and filling na values with mean
    for i in final_df:
        if i in rare_feat or i in bin_feat:
            continue
        final_df = final_df.fillna(final_df.mean())
        non_bin_feat.append(i)
        
#     display(final_df.head())
    return final_df

In [9]:
# Second Matrix

df_static = df.loc[df['time'] == '00:00', :].copy()
static_vars = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']
df_static.drop('time', axis=1, inplace=True)
df_static = df_static.loc[df_static['parameter'].isin(static_vars)]
df_static = df_static.groupby(['recordid', 'parameter'])[['value']].last()
df_static.reset_index(inplace=True)
df_static = df_static.pivot(index='recordid', columns='parameter', values='value')
df_static

for c in df_static.columns:
    df_static[c] = pd.to_numeric(df_static[c])

for c in df_static.columns:
    x = df_static[c]
    if c == 'Height':
        idx = x < 0
        df_static.loc[idx, c] = np.nan
    elif c == 'Weight':
        idx = x < 0
        df_static.loc[idx, c] = np.nan
df_static.head()

df2 = df_static.copy()
df2.drop('RecordID', axis=1, inplace=True)
df2.head()

for x in rare_feat:
    id = specialFeature(x)
    df2[x] = 0
    for i in id:
        for row in df2.index:
            if row == i:
                df2.loc[row, x] = 1
df2.head()

idx = df['parameter'].isin(nor_feat)
df3 = df.loc[idx, :].copy()
df3[['hour','min']] = df3.time.str.split(':', expand=True)
df3["hour"] = pd.to_numeric(df3["hour"])
df3["value"] = pd.to_numeric(df3["value"])
bins = [0, 12, 24, 36, 48]
labels = ['0', '12', '24', '36']
df3 = df3.groupby(['recordid', pd.cut(df3.hour, bins=bins, labels=labels), 'parameter'])[['value']].mean()
df3

for n in nor_feat:    
    df2[n +'0'] = np.nan
    df2[n +'12'] = np.nan
    df2[n +'24'] = np.nan
    df2[n +'36'] = np.nan
df2.head()

for index, row in df3.iterrows():
    recordId = index[0]
    hour = index[1]
    parameter = index[2]
    df2.loc[recordId, parameter+hour] = row["value"]
df2.head()

parameter,Age,Gender,Height,ICUType,Weight,Bilirubin,ALT,AST,ALP,Albumin,TroponinT,TroponinI,Cholesterol,MechVent,HR0,HR12,HR24,HR36,MAP0,MAP12,MAP24,MAP36,SysABP0,SysABP12,SysABP24,SysABP36,DiasABP0,DiasABP12,DiasABP24,DiasABP36,Urine0,Urine12,Urine24,Urine36,Weight0,Weight12,Weight24,Weight36,NISysABP0,NISysABP12,NISysABP24,NISysABP36,NIDiasABP0,NIDiasABP12,NIDiasABP24,NIDiasABP36,NIMAP0,NIMAP12,NIMAP24,NIMAP36,Temp0,Temp12,Temp24,Temp36,GCS0,GCS12,GCS24,GCS36,RespRate0,RespRate12,RespRate24,RespRate36,FiO20,FiO212,FiO224,FiO236,MechVent0,MechVent12,MechVent24,MechVent36,pH0,pH12,pH24,pH36,PaO20,PaO212,PaO224,PaO236,PaCO20,PaCO212,PaCO224,PaCO236,HCT0,HCT12,HCT24,HCT36,K0,K12,K24,K36,Creatinine0,Creatinine12,Creatinine24,Creatinine36,Platelets0,Platelets12,Platelets24,Platelets36,BUN0,BUN12,BUN24,BUN36,HCO30,HCO312,HCO324,HCO336,Mg0,Mg12,Mg24,Mg36,Na0,Na12,Na24,Na36,Glucose0,Glucose12,Glucose24,Glucose36,WBC0,WBC12,WBC24,WBC36,SaO20,SaO212,SaO224,SaO236,Lactate0,Lactate12,Lactate24,Lactate36
recordid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
132539,54.0,0.0,NaN,4.0,NaN,0,0,0,0,0,0,0,0,0,65.454545,66.090909,78.833333,78.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.000000,86.250000,173.333333,308.333333,NaN,NaN,NaN,NaN,111.700000,104.125000,111.333333,120.875000,50.600000,45.125000,44.333333,56.125000,70.967000,64.792500,66.666667,77.708750,37.833333,37.100000,38.066667,37.766667,15.000000,15.0,14.666667,15.00,16.545455,15.545455,17.857143,19.636364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.600,NaN,30.30,NaN,4.4,NaN,4.00,NaN,0.8,NaN,0.7,NaN,221.000000,NaN,185.0,NaN,13.0,NaN,8.0,NaN,26.0,NaN,28.0,NaN,1.5,NaN,1.90,NaN,137.0,NaN,136.0,NaN,205.0,NaN,115.0,NaN,11.2,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132540,76.0,1.0,175.3,2.0,76.0,0,0,0,0,0,0,0,0,1,87.076923,80.055556,75.307692,73.636364,75.740741,76.111111,78.384615,80.111111,106.714286,114.222222,119.923077,123.222222,60.892857,56.888889,57.846154,58.222222,158.933333,103.181818,170.714286,187.500000,NaN,80.6,80.6,81.236364,NaN,111.750000,115.142857,107.333333,NaN,57.500000,61.000000,45.666667,NaN,75.580000,79.047143,66.223333,36.688462,37.500000,36.850000,36.800000,10.800000,15.0,14.666667,14.25,NaN,NaN,NaN,NaN,0.56,NaN,NaN,NaN,1.0,NaN,NaN,NaN,7.385,7.40,NaN,7.385,226.25,NaN,NaN,111.0,37.0,NaN,NaN,45.0,27.625,28.900000,30.70,29.45,NaN,4.30,NaN,3.5,0.8,1.2,NaN,1.3,190.333333,187.0,NaN,135.0,16.0,18.0,NaN,21.0,21.0,22.0,NaN,24.0,3.1,1.9,NaN,2.1,NaN,139.0,NaN,135.0,NaN,105.0,NaN,146.0,7.4,13.1,NaN,13.3,98.0,97.0,NaN,95.0,NaN,NaN,NaN,NaN
132541,44.0,0.0,NaN,3.0,56.7,1,1,1,1,1,0,0,0,1,90.000000,83.250000,87.666667,71.166667,NaN,NaN,100.000000,87.250000,NaN,NaN,137.500000,121.750000,NaN,NaN,75.500000,64.333333,111.818182,185.500000,150.000000,58.800000,56.7,56.7,56.7,56.700000,132.812500,136.750000,124.500000,NaN,78.750000,80.583333,76.000000,NaN,96.767500,99.305000,92.163750,NaN,37.825000,37.233333,38.300000,37.833333,7.333333,6.0,5.000000,5.00,NaN,NaN,NaN,NaN,0.75,0.5,0.46,0.4,1.0,1.0,1.0,1.0,NaN,7.51,7.49,NaN,NaN,65.0,157.666667,NaN,NaN,37.0,35.0,NaN,28.500,26.700000,28.85,29.40,3.3,8.60,2.85,3.7,0.4,0.3,NaN,0.3,72.000000,84.0,NaN,113.0,8.0,3.0,NaN,3.0,24.0,26.0,NaN,25.0,1.9,1.3,1.85,1.7,137.0,140.0,NaN,138.0,141.0,119.0,NaN,143.0,4.2,3.7,NaN,6.2,NaN,95.0,NaN,NaN,1.3,1.9,0.9,NaN
132543,68.0,1.0,180.3,3.0,84.6,1,1,1,1,1,0,0,0,0,72.238095,72.500000,63.769231,74.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600.000000,400.000000,675.000000,600.000000,84.6,84.6,84.6,84.600000,122.571429,122.090909,117.750000,121.545455,68.285714,61.636364,61.500000,66.181818,86.381429,81.788182,80.250000,84.637273,35.966667,36.433333,36.133333,36.333333,14.500000,15.0,15.000000,15.00,16.100000,14.916667,13.384615,16.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.300,36.850000,36.20,36.30,4.2,NaN,3.80,NaN

In [ ]:
# Reading features
# fold1 = put_single_into_dataframe(read_text("../Project_Data/Fold1"))
# fold2 = put_single_into_dataframe(read_text("../Project_Data/Fold2"))
# fold3 = put_single_into_dataframe(read_text("../Project_Data/Fold3"))
# fold4 = put_single_into_dataframe(read_text("../Project_Data/Fold4"))
# df_feat = fold1.copy()
# df_feat = df_feat.append(fold2)
# df_feat = df_feat.append(fold3)
# df_feat = df_feat.append(fold4)
# df_feat.head()

<h1>Model Building</h1>
<p>We develop models for our design matrix.</p>
<ul> 
    <li>First, in the cell below, we make use of the first design matrix above to pass in as input to the models. It mannually does cross validation on the 3 folds and validation on the remaining fold. Included is 3 model (Linear, DecisionTree, MLP) pipelines utilising diffrent feature selectors </li>
    <li></li>
    <li></li>
</ul>

In [10]:
bin_feat = ["Gender", "Coronary Care Unit", "Cardiac Surgery Recovery Unit", "Medical ICU", "Surgical ICU"]
non_bin_feat = []

scaler = ColumnTransformer(
    remainder = 'passthrough',
    transformers=[
        ('num', StandardScaler(), non_bin_feat)])

parameters = {'VarianceThreshold':[0,0.5], 'PCA':[60,70],
              'SelectKBest':[80, 90],
              'DecisionTreeRegressor_min_samples_leaf':[1,3],
              'MLPRegressor_hiddenLayer':[(30,30,), (100,)]
             }

all_list = [1,2,3,4]
for i in range(1,5):
    print("Testing on Fold", i)
    x_train_df = pd.DataFrame()
    y_train_df = pd.DataFrame()
    
    # Getting train data set up
    for j in [x for x in all_list if x != i]: 
        string = "../Project_Data/Fold"+str(j)
        y_file = "../Project_Data/Fold"+str(j)+"_Outcomes.csv"
        x_train_df = x_train_df.append(put_single_into_dataframe(read_text(string)))
        y_train_df = y_train_df.append(pd.DataFrame(read_ans(y_file), columns=['recordid', 'days_in_hospital', 'mortality']))
    x_train_df = preprocess_x_for_design_matrix_1(x_train_df)
    y_train_df = y_train_df.drop(['recordid','mortality'], axis=1)
    for index, row in y_train_df.iterrows():
        if row['days_in_hospital'] == -1:
            row['days_in_hospital'] = 2
        
    # Getting test data set up
    x_test_df = put_single_into_dataframe(read_text("../Project_Data/Fold"+str(i)))
    x_test_df = preprocess_x_for_design_matrix_1(x_test_df)
    y_test_df = pd.DataFrame(read_ans("../Project_Data/Fold"+str(i)+"_Outcomes.csv"), columns=['recordid', 'days_in_hospital', 'mortality'])
    y_test_df = y_test_df.drop(['recordid','mortality'], axis=1)
    for index, row in y_test_df.iterrows():
        if row['days_in_hospital'] == -1:
            row['days_in_hospital'] = 2
    
    # Creating model and parameters to try out
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            est = Pipeline(steps=[
            ('scaler', scaler),
            ('f_selecter', VarianceThreshold(threshold = nThreshold)),
            ('dim_reducer', PCA(n_components = nN_components)),
            ('classifier', LinearRegression())])
            est.fit(x_train_df, y_train_df)
            prediction = est.predict(x_test_df)
            print ( "Using LinearRegression --> VarianceThreshold: {} and PCA: {} has a score of {}".format(nThreshold, nN_components, mean_squared_error(y_test_df, prediction)) )
    for nThreshold in parameters["SelectKBest"]:
        for nN_components in parameters["PCA"]:
            for min_samples_leaf in parameters["DecisionTreeRegressor_min_samples_leaf"]:
                est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', SelectKBest(k = 90)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', DecisionTreeRegressor(min_samples_leaf = min_samples_leaf))])
                est.fit(x_train_df, y_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using DecisionTreeRegressor({}) --> SelectKBest: 90 and PCA: {} has a score of {}".format(min_samples_leaf, nN_components, mean_squared_error(y_test_df, prediction)) )
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            for hiddenLayer in parameters["MLPRegressor_hiddenLayer"]:
                est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', VarianceThreshold(threshold = nThreshold)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', MLPRegressor(hidden_layer_sizes = hiddenLayer, learning_rate_init = 0.01))])
                est.fit(x_train_df, y_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using MLPRegressor({}) --> SelectKBest: 90 and PCA: {} has a score of {}".format(hiddenLayer, nN_components, mean_squared_error(y_test_df, prediction)) )
    print(end="\n\n")


Testing on Fold 1
Using LinearRegression --> VarianceThreshold: 0 and PCA: 60 has a score of 183.22308625288238
Using LinearRegression --> VarianceThreshold: 0 and PCA: 70 has a score of 182.634760530275
Using LinearRegression --> VarianceThreshold: 0.5 and PCA: 60 has a score of 183.68860550958735
Using LinearRegression --> VarianceThreshold: 0.5 and PCA: 70 has a score of 185.34983630223107


C:\Users\samue\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\samue\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [87 88 89 90 91 92 93 94 95] are constant.
  UserWarning)
C:\Users\samue\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Using DecisionTreeRegressor(1) --> SelectKBest: 90 and PCA: 60 has a score of 311.987


C:\Users\samue\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\samue\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [87 88 89 90 91 92 93 94 95] are constant.
  UserWarning)
C:\Users\samue\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Using DecisionTreeRegressor(3) --> SelectKBest: 90 and PCA: 60 has a score of 304.87670722222225


...

C:\Users\samue\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using MLPRegressor((30, 30)) --> SelectKBest: 90 and PCA: 70 has a score of 292.9557712617733


C:\Users\samue\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using MLPRegressor((100,)) --> SelectKBest: 90 and PCA: 70 has a score of 286.15194869752565


Testing on Fold 2


KeyboardInterrupt: 